In [1]:
import pandas as pd
import numpy as np

all_response_df = pd.read_csv('../data/tcga/PDR_data/tcga698_single_drug_response_df.csv',index_col=1)
# response_type_df = pd.read_csv('../data/tcga/PDR_data/tcga667_single_drug_response_type_df.csv',index_col=1)

Tumor_type_list = [ "brca", "cesc", "coad", "gbm", "hnsc", "kich" ,"kirc", "kirp" , "lgg" ,
              "luad", "lusc",  "paad",  "read", "sarc", "skcm", "stad"] #"ucec",
            # "lgg","blca","ov","prad","kich","kirp","lihc", "esca","meso","ucs","acc"] #20+4
# tumor_type = [element.upper() for element in Tumor_type_list][1]

for tumor_type in [element.upper() for element in Tumor_type_list]:
    response_df = all_response_df.loc[all_response_df['tcga_project'] == tumor_type]
    response_df = response_df[['days_to_new_tumor_event_after_initial_treatment','smiles','cid']]
        
    response_df['label'] = pd.qcut(response_df['days_to_new_tumor_event_after_initial_treatment'],2,labels = range(2))
    response_df.to_csv(f"../data/preprocessed_dat/tcga_clinical_data/{tumor_type}_{response_df.shape[0]}.csv")


In [6]:
#for KG
import pandas as pd
import numpy as np

all_response_df = pd.read_csv('../data/tcga/PDR_data/tcga698_single_drug_response_df.csv',index_col=1)
# response_type_df = pd.read_csv('../data/tcga/PDR_data/tcga667_single_drug_response_type_df.csv',index_col=1)

Tumor_type_list = [ "brca", "cesc", "coad", "gbm", "hnsc", "kich" ,"kirc", "kirp" , "lgg" ,
              "luad", "lusc",  "paad",  "read", "sarc", "skcm", "stad"] #"ucec",
            # "lgg","blca","ov","prad","kich","kirp","lihc", "esca","meso","ucs","acc"] #20+4
# tumor_type = [element.upper() for element in Tumor_type_list][1]

Tumor_type_list = [ "brca", "cesc", "coad", "gbm", "hnsc", "kich" ,"kirc", "kirp" , "lgg" ,
              "luad", "lusc",  "read", "sarc", "skcm", "stad"]

for tumor_type in [element.upper() for element in Tumor_type_list]:
    response_df = all_response_df.loc[all_response_df['tcga_project'] == tumor_type]
    response_df = response_df[['days_to_new_tumor_event_after_initial_treatment','drug_name']]
        
    response_df['label'] = pd.qcut(response_df['days_to_new_tumor_event_after_initial_treatment'],2,labels = range(2))
    response_df.reset_index(inplace=True)
    del response_df['days_to_new_tumor_event_after_initial_treatment']
    response_df.columns = ["Sample_id","Drug_name","Label"]
    response_df.to_csv(f"../data/preprocessed_dat/tcga_clinical_data/{tumor_type}_{response_df.shape[0]}.csv")


In [10]:
#for KG
all_response_df = pd.read_csv('../data/tcga/PDR_data/tcga698_single_drug_response_df.csv',index_col=1)
response_df = all_response_df[['drug_name','smiles','cid']]
response_df.index = range(response_df.shape[0])
response_df.drop_duplicates(inplace=True)
response_df.index = range(response_df.shape[0])
response_df.columns = ["Drug_name","Smiles","Cid"]
response_df
response_df.to_csv(f"../data/preprocessed_dat/tcga_clinical_data/TCGA_drug_info.csv")

/tmp/ipykernel_1183435/3823672090.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  response_df.drop_duplicates(inplace=True)


## 调试得到所有癌种的Zero-shot dataset

In [1]:
import data

Tumor_type_list = ["tcga", "blca", "brca", "cesc", "coad", "gbm", "hnsc", "kich", "kirc", 
            "kirp", "lgg", "lihc", "luad", "lusc", "ov", "paad", "prad", "read", "sarc", "skcm", "stad", #"ucec",
            "esca","meso","ucs","acc"] 

gex_features_df,CCL_tumor_type,all_ccle_gex,all_patient_gex = data.get_pretrain_dataset(
        patient_tumor_type = "brca",
        CCL_type = "all_CCL",
        tumor_type = "BRCA",
        tcga_construction = "raw",
        CCL_construction = "raw",
        gene_num = 1000,select_gene_method = "Percent_sd"
        )

/opt/conda/envs/python3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting generating pretrain dataset...
Step1 load in dim: patient(10717); ccle(1917)
Step2 after locate dim: patient(1099); ccle(1305)
Define CCL_tumor_type: all_CCL
Step3 after pseudo construct dim: patient(1099); ccle(1305)
Step4 after select HVGs: patient_gene(1000):ccle_gene(1000):union_gene(1672)
Step5 after combination dim: 2404 samples  *  1672 genes



In [2]:
select_Tumor_type_list = Tumor_type_list[2:20]
select_Tumor_type_list = [element.upper() for element in Tumor_type_list[2:20]]
print(f'Zero-shot to these TT: {select_Tumor_type_list}')

Zero-shot to these TT: ['BRCA', 'CESC', 'COAD', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'OV', 'PAAD', 'PRAD', 'READ', 'SARC', 'SKCM']


In [3]:
import data
labeled_tcga_dataloader_all_TT = data.get_all_tcga_ZS_dataloaders(all_patient_gex, 
                    batch_size=64,
                    Tumor_type_list=select_Tumor_type_list,
                    label_type = "PFS",q=2)

In [4]:
labeled_tcga_dataloader_all_TT

<generator object get_all_tcga_ZS_dataloaders at 0x7fae7956e7b0>

In [6]:
for labeled_tcga_dataloader,TT in labeled_tcga_dataloader_all_TT:
    for x_gex ,x_smiles, y_batch in labeled_tcga_dataloader:
        print(x_smiles)

: 

ValueError: not enough values to unpack (expected 3, got 1)